# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
from __future__ import annotations
import json
import os
import os.path as _p
import hashlib
from fs.osfs import OSFS
from pathlib import Path
from schematized_config.core import ConfigValidator
from blobslayer.DotenvSchema import DotenvSchema
from blobslayer.FileMetadataSchema import FileMetadataSchema
from typing import ClassVar
import blobslayer

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

def recursive_substitute(env_vars: dict) -> dict:
    """
    Recursively substitute environment variables in the values of the env_vars dictionary.
    """
    while True:
        changed = False
        for key, value in env_vars.items():
            substituted_value = value
            for k, v in env_vars.items():
                # Substitute the value if the key is found in the value string
                if f"${k}" in substituted_value:
                    substituted_value = substituted_value.replace(f"${k}", v)
                    changed = True
            
            env_vars[key] = substituted_value

        # If no substitutions were made in this pass, break the loop
        if not changed:
            break

    return env_vars

In [ ]:
#| hide

def _load_config(dotenv_path='.env'):
    if _p.exists(dotenv_path):
        with open(Path(blobslayer.__file__).parent.parent / 'schemas' / 'dotenv.schema.json') as ifile:
            json_schema = json.load(ifile)
        validated_dotenv = ConfigValidator.load_dotenv(
            json_schema=json_schema,
            dotenv_path=dotenv_path,
            storage_driver=OSFS(os.getcwd()),
        )  # use defaults of .env and CONFIG_VALIDATOR_JSON_SCHEMA
        return DotenvSchema(**recursive_substitute(validated_dotenv))

def _notebook_load_default_config():
    config = None
    try:
        # local block to load .env from project root
        cwd = os.getcwd()
        os.chdir('..')
        config = _load_config()
        os.chdir(cwd)
    except Exception as e:
        print('failed to load env:')
        print(e)
        os.chdir(cwd)
    return config

Config = _notebook_load_default_config()
assert Config is not None

In [ ]:
#| export

def _ensure_storage_directories_exist(config: DotenvSchema):    
    Path(config.STORAGE_DIRECTORY).mkdir(parents=True, exist_ok=True)
    Path(config.BLOBS_DIRECTORY).mkdir(parents=True, exist_ok=True)
    Path(config.METADATA_DIRECTORY).mkdir(parents=True, exist_ok=True)


In [ ]:
#| export

def sha256sum(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()

def to_split_path(hash_path: str) -> str:
    return f'{hash_path[:2]}/{hash_path[2:4]}/{hash_path[4:]}'

def save_blob(data: bytes, blobs_directory: str=None) -> str:
    blobs_directory = blobs_directory or Config.BLOBS_DIRECTORY
    assert blobs_directory is not None
    blob_key = sha256sum(data)
    with OSFS(blobs_directory) as myfs:
        blob_path = to_split_path(blob_key)
        blob_split_dir, _ = _p.split(blob_path)
        myfs.makedirs(blob_split_dir, recreate=True)
        if myfs.exists(blob_path):
            return blob_key
        with myfs.open(blob_path, 'wb') as ofile:
            ofile.write(data)
    return blob_key

def get_blob(blob_key: str, blobs_directory: str=None):  # returns a file-like
    blobs_directory = blobs_directory or Config.BLOBS_DIRECTORY
    assert blobs_directory is not None
    blob_path = to_split_path(blob_key)
    with OSFS(blobs_directory) as myfs:
        return myfs.open(blob_path, 'rb')

def to_metadata_path(filepath: str):
    return f'{filepath}.meta.json'

def write_file_as_blob(
        filepath: str,
        data: bytes,
        metadata_directory: str=None,
        blobs_directory: str=None
    ) -> str:
    metadata_directory = metadata_directory or Config.METADATA_DIRECTORY
    blobs_directory = blobs_directory or Config.BLOBS_DIRECTORY
    assert metadata_directory is not None and blobs_directory is not None
    blob_key = save_blob(data, blobs_directory)
    file_metadata = FileMetadataSchema(
        blobKey = blob_key,
    )
    with OSFS(metadata_directory) as myfs:
        metadata_path = to_metadata_path(filepath)
        metadata_sub_dir, _ = _p.split(metadata_path)
        myfs.makedirs(metadata_sub_dir, recreate=True)
        with myfs.open(metadata_path, 'w') as ofile:
            ofile.write(json.dumps(dict(file_metadata)))
    return blob_key

def get_file_metadata(
        filepath: str,
        metadata_directory: str=None,
) -> dict:
    metadata_directory = metadata_directory or Config.METADATA_DIRECTORY
    assert metadata_directory is not None
    with OSFS(metadata_directory) as myfs:
        metadata_path = to_metadata_path(filepath)
        if not myfs.exists(metadata_path):
            return None
        with myfs.open(metadata_path) as ifile:
            return json.load(ifile)
        
def set_file_metadata(
        filepath: str,
        metadata,
        metadata_directory: str=None,
):
    metadata_directory = metadata_directory or Config.METADATA_DIRECTORY
    assert metadata_directory is not None
    with OSFS(metadata_directory) as myfs:
        metadata_path = to_metadata_path(filepath)
        with myfs.open(metadata_path, 'w') as ofile:
            ofile.write(json.dumps(dict(metadata)))

def read_file_blob(
        filepath: str,
        metadata_directory: str=None,
        blobs_directory: str=None
    ):
    metadata_directory = metadata_directory or Config.METADATA_DIRECTORY
    blobs_directory = blobs_directory or Config.BLOBS_DIRECTORY
    assert metadata_directory is not None and blobs_directory is not None
    metadata = get_file_metadata(filepath, metadata_directory)
    assert metadata is not None
    file_metadata = FileMetadataSchema(**metadata)
    return get_blob(file_metadata.blobKey)

In [ ]:
#| export

class BlobSlayer(DotenvSchema):

    _default_instance: ClassVar['BlobSlayer'] = None
    
    @classmethod
    def get_default_instance(cls, default_conf: DotenvSchema=None):
        if cls._default_instance is None:
            default_conf = default_conf or cls()
            cls._default_instance = cls(**recursive_substitute(dict(default_conf)))
            _ensure_storage_directories_exist(cls._default_instance)
        return cls._default_instance
    
    def get_metadata(self, filepath: str) -> dict:
        return get_file_metadata(
            filepath,
            self.METADATA_DIRECTORY)

    def set_metadata(self, filepath: str, metadata):
        return set_file_metadata(
            filepath,
            metadata,
            self.METADATA_DIRECTORY)

    def save_file(self, filepath: str, data: bytes):
        return write_file_as_blob(
            filepath,
            data,
            metadata_directory=self.METADATA_DIRECTORY,
            blobs_directory=self.BLOBS_DIRECTORY)
    
    def load_file(self, filepath: str):
        return read_file_blob(
            filepath,
            metadata_directory=self.METADATA_DIRECTORY,
            blobs_directory=self.BLOBS_DIRECTORY)

In [ ]:
#| hide

bs = BlobSlayer.get_default_instance(_notebook_load_default_config())

file_path = 'foo/bar/baz.txt'
file_content = b'lorem ipsum'
my_file = bs.save_file(file_path, file_content)
with (bs.load_file(file_path)) as ifile:
    test_eq(ifile.read(), file_content)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()